<a href="https://colab.research.google.com/github/marienbaptiste/IBM-Capstone/blob/master/Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center">Segmenting and Clustering Neighbourhoods in Toronto</h1>


*The whole assigment is in this notebook*



##Part 1

In [0]:
#Libraries
import requests
import pandas as pd
import numpy as np

**1.   Scraping data with BeautifulSoup**

In [0]:
# Make the request to a url
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# Create soup from content of request
c = r.content

from bs4 import BeautifulSoup

soup = BeautifulSoup(c) #soup in now the html output

In [0]:
# Inspecting the code, we can find that our data in wrapped in a "wikitable sortable jquery-tablesorter", let's isolate that
neigh_table = soup.find('table',{'class':'wikitable sortable'})

# Our data in <td> is nested into <tr>
table_rows = neigh_table.find_all('tr')

#Let's append it all in a list called data
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

### **2.   The dataframe will consist of three columns: PostalCode, Borough, and Neighbourhood**

In [4]:
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])

#From a first inspection, it looks like we created rows with empty values
print((~df['PostalCode'].isnull()).value_counts())

df = df[~df['PostalCode'].isnull()]  #Filter the artifact at the beginning (empty row)

#Let's check the cleaned up result
print(df.shape)
df.head()

True     287
False      1
Name: PostalCode, dtype: int64
(287, 3)


,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### **3.   Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.**

In [5]:
#Droping the rows in "Borough" containing the string "Not assigned"

#Building the condition
indexNa = df[ df['Borough'] == 'Not assigned' ].index
 
#Delete these row indexes from dataFrame
df.drop(indexNa, inplace=True)

#Reset the index to 0
df.reset_index(drop=True, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


### **4&5.   If more than one neighbourhood exists per postal code, it should be combined with a comma**

In [6]:
#Finding evidence of those duplicates
def find_dupes():
	return str(df[df.groupby('PostalCode')['Neighbourhood'].transform('nunique') > 1].shape[0])

print ("Number of duplicates before processing: " + find_dupes()) #163 duplicates found

#Proceed to aggregate neighbourhood sharing PostalCode
df=df.groupby(['PostalCode','Borough'], sort=False).agg(', '.join)
df.reset_index(inplace=True) #Reset the index to 0

#Checking
print ("Number of duplicates after processing: " + find_dupes()) #0 duplicate found, nice!
df.head() #M5A doesn't have any duplicate, this is confirmed when inspecting the Wikipedia page

Number of duplicates before processing: 163
Number of duplicates after processing: 0


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


### **6.  When a cell has a borough but a Not assigned neighbourhood, then the neighbourhood will be the same as the borough.**

In [7]:
#Conditional return
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned',df['Borough'], df['Neighbourhood'])
df.head() #Observing that M7A got the right value

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


### **7.  In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe**

In [8]:
#OK
df.shape

(103, 3)

##Part 2

In [9]:
#Libraries
!pip install geocoder #No conda on Google Colab...
import geocoder # import geocoder

#Utility function
def get_loc(postal_code):
     # initialize your variable to None
     lat_lng_coords = None
     # loop until you get the coordinates
     while(lat_lng_coords is None):
       g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
       lat_lng_coords = g.latlng
       
     print(lat_lng_coords)
     latitude = lat_lng_coords[0]
     longitude = lat_lng_coords[1]
     return latitude,longitude

     |████████████████████████████████| 102kB 2.4MB/s 


### **1.   Import Geocodes**

In [10]:
#We built the function above, we just have to iterate
#for i in range(0,len(df)):
#    df['Latitude'][i],df['Longitude'][i]=get_loc(df.iloc[i]['PostalCode'])

#Falling back to the cvs file, as the API is broken
lat_lon_data = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon_data.head() #We can observe that we must merge the tables

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### **2.   Fixing before join**

In [11]:
#Postal Code in the newly imported csv is called PostalCode in our Dataframe, let's rename that
lat_lon_data.rename(columns={'Postal Code':'PostalCode'},inplace=True)
lat_lon_data.head() #We can observe that we must merge the tables

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### **3.   Merging**

In [12]:
#And now the merging using an inner joint, which is default but making it explicit
df_merged = pd.merge(df,lat_lon_data,on='PostalCode', how='inner')
#Checking, rows should be conserved
df_merged.shape #103, rows count has been untouched

(103, 5)

### **4.   Output**

In [13]:
#Final result
df_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


##Part 3

In [14]:
#Libraries

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install folium
import folium # mapping library

from getpass import getpass # to hide secret key

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

### **1.   Explore and cluster the neighbourhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto.**

In [15]:
#Generating a subset of our dataframe based on filter
df_toronto = df_merged[df_merged['Borough'].str.contains('Toronto')]
df_toronto.reset_index(drop=True, inplace=True) #For navigation ease
df_toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


### **2.   Replicate the same analysis we did to the New York City data. It is up to you. First we shall plot the neighbourhood**

In [16]:
#Let's get the geographical coordinates of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [17]:
#Visualizing all the Neighbourhoods of the above data frame using Folium

#Create a map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

#Add markers to map
for lat, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Picture for reference as it is not rendered on GitHub
![mockup picture](https://raw.githubusercontent.com/marienbaptiste/IBM-Capstone/master/toronto_plainplots.png)

### **3.   Foursquare credentials and version (This is done to protect the values in Colab)**

In [18]:
CLIENT_ID = 'SR4IUUCSUAPSQMPHRMJMFL4E3K1PZNNJGGTAMKCANJ1V4XSY' # your Foursquare ID
CLIENT_SECRET = getpass('Enter the client secret value: ') # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Enter the client secret value: ··········


### **4.   Let's explore the first neighbourhood in our dataframe.**

In [19]:
df_toronto.loc[0, 'Neighbourhood']

'Harbourfront'

In [20]:
neighbourhood_latitude = df_toronto.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = df_toronto.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = df_toronto.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


### **5.   Now, let's get the top 100 venues that are in Harbourfront within a radius of 500 meters.**

In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# type your answer here
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=SR4IUUCSUAPSQMPHRMJMFL4E3K1PZNNJGGTAMKCANJ1V4XSY&client_secret=RLWCOIU5E1JZDMUCH4EIHHDHN5IKQAN4IWWUSHE3ZVKRI5HG&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [0]:
#Fetching, careful with daily limit
results = requests.get(url).json()

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### **6.   Now we are ready to clean the json and structure it into a *pandas* dataframe.**

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

48 venues were returned by Foursquare.


### **7.   Let's create a function to repeat the same process to all the neighbourhoods in Manhattan.**

In [0]:
#Careful! This consumes API usage
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
#Main call, most consuming bit
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Harbourfront
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian P

In [28]:
#Checking the return
print(toronto_venues.shape)
toronto_venues.head()

(1698, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


### **8.   Let's check how many venues were returned for each neighbourhood.**

In [29]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,58,58,58,58,58,58
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,83,83,83,83,83,83
"Chinatown, Grange Park, Kensington Market",95,95,95,95,95,95
Christie,17,17,17,17,17,17


In [30]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 230 uniques categories.


### **9.   Analyze Each Neighborhood**

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,...,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### **10.   Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [32]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,...,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.03,0.0,0.0,0.0,0.000000,0.0,0.03,0.0,0.000000,0.000000,0.0,0.000000,0.030000,0.0,0.040000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.02,0.0,0.01,0.030000,0.000000,0.0,0.0,0.03,...,0.0,0.000000,0.000000,0.0,0.010000,0.0,0.0,0.0,0.0,0.000000,0.01,0.000000,0.0,0.000000,0.0,0.040000,0.0,0.0,0.03,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.040000,0.01,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.0,0.000000
1,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,0.0,0.0,0.017241,0.0,0.00,0.0,0.000000,0.017241,0.0,0.017241,0.051724,0.0,0.000000,0.0,0.017241,0.017241,0.0,0.034483,0.0,0.0,0.017241,0.0000,0.00,0.0,0.00,0.017241,0.000000,0.0,0.0,0.00,...,0.0,0.017241,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.017241,0.0,0.000000,0.0,0.034483,0.0,0.0,0.00,0.0,0.0,0.017241,0.0,0.0,0.017241,0.0,0.017241,0.00,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.00,0.0,0.0,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.045455,0.0,0.045455,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.00,0.0,0.00,0.090909,0.000000,0.0,0.0,0.00,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.045455,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.045455
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.055556,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.00,0.0,0.00,0.000000,0.055556,0.0,0.0,0.00,...,0.0,0.000000,0.055556,0.0,0.055556,0.0,0.0,0.0,0.0,0.055556,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.055556
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.062500,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0625,0.00,0.0,0.00,0.000000,0.000000,0.0,0.0,0.00,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000


In [33]:
#New size
toronto_grouped.shape

(38, 231)

### **11.   Let's print each neighborhood along with the top 5 most common venues**

In [59]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1              Bakery  0.05
2  Seafood Restaurant  0.03
3        Cocktail Bar  0.03
4      Farmers Market  0.03


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0          Breakfast Spot  0.09
1             Coffee Shop  0.09
2                    Café  0.09
3             Yoga Studio  0.05
4  Furniture / Home Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2                 Spa  0.06
3       Garden Center  0.06
4              Garden  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airpo

In [0]:
#As a Dataframe:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Asian Restaurant,Bakery,American Restaurant,Sushi Restaurant,Burger Joint
1,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Beer Bar,Steakhouse,Cheese Shop,Seafood Restaurant,Café,Farmers Market,Breakfast Spot
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Grocery Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Plane,Bar,Harbor / Marina,Coffee Shop,Sculpture Garden,Boat or Ferry,Airport


### **12.   Clustering**

In [62]:
#Run k-means to cluster the neighborhood into 5 clusters.
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [63]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Café,Mexican Restaurant,Breakfast Spot,Theater,Yoga Studio,Spa
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Bakery,Cosmetics Shop,Fast Food Restaurant,Bookstore,Restaurant,Diner,Ramen Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Clothing Store,Cosmetics Shop,Bakery,Gastropub,Cocktail Bar
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Trail,Health Food Store,Pub,Neighborhood,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop,Event Space
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Cocktail Bar,Beer Bar,Steakhouse,Cheese Shop,Seafood Restaurant,Café,Farmers Market,Breakfast Spot


### **13.   Mapping**

In [64]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Picture for reference as it is not rendered on GitHub
![mockup picture](https://raw.githubusercontent.com/marienbaptiste/IBM-Capstone/master/toronto_clusterplots.png)

### **14.   Examining**

*We can see that overall the city has a pretty similar profile regarding repartition in venues, only few places are presenting a different one, especially on the East and on the North*

### Cluster *1*

In [65]:
#Examining discriminating venues in Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Café,Mexican Restaurant,Breakfast Spot,Theater,Yoga Studio,Spa
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Bakery,Cosmetics Shop,Fast Food Restaurant,Bookstore,Restaurant,Diner,Ramen Restaurant
2,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Clothing Store,Cosmetics Shop,Bakery,Gastropub,Cocktail Bar
4,Downtown Toronto,0,Coffee Shop,Bakery,Cocktail Bar,Beer Bar,Steakhouse,Cheese Shop,Seafood Restaurant,Café,Farmers Market,Breakfast Spot
5,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Burger Joint,Sandwich Place,Bubble Tea Shop,Gym / Fitness Center,Japanese Restaurant,Salad Place
6,Downtown Toronto,0,Grocery Store,Café,Park,Convenience Store,Candy Store,Baby Store,Diner,Athletics & Sports,Restaurant,Nightclub
7,Downtown Toronto,0,Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Asian Restaurant,Bakery,American Restaurant,Sushi Restaurant,Burger Joint
8,West Toronto,0,Bakery,Pharmacy,Supermarket,Gym / Fitness Center,Music Venue,Café,Middle Eastern Restaurant,Brewery,Brazilian Restaurant,Bar
9,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Fried Chicken Joint,Brewery,Italian Restaurant,Scenic Lookout,Bakery
10,West Toronto,0,Bar,Coffee Shop,Asian Restaurant,Men's Store,Restaurant,Vietnamese Restaurant,French Restaurant,New American Restaurant,Café,Bakery


### Cluster *2*

In [66]:
#Examining discriminating venues in Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Garden,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Cluster *3*

In [67]:
#Examining discriminating venues in Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,2,Playground,Tennis Court,Park,Restaurant,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
32,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### Cluster *4*

In [68]:
#Examining discriminating venues in Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,3,Park,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


### Cluster *5*

In [69]:
#Examining discriminating venues in Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,4,Trail,Health Food Store,Pub,Neighborhood,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop,Event Space
